In [1]:
from dhanhq import dhanhq, marketfeed
import talib
import numpy as np
import pandas as pd
import time
import datetime
import os
import logging

In [3]:
with open("token.txt", "r") as file:
    # Read the entire contents of the file
    token = file.read()
client_id = "1102249582"
access_token = token
dhan = dhanhq(client_id,access_token)

masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', nrows=1)
masterlist = pd.read_csv('https://images.dhan.co/api-data/api-scrip-master.csv', usecols=masterlist.columns, low_memory=False)


logging.basicConfig(filename="logfile.txt", 
                    level=logging.INFO, 
                    format="%(asctime)s - %(levelname)s - %(message)s")


In [5]:
NIFTY_PREMIMUM = 25100

filtered_ce_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-07-24 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'CE') 
]
filtered_ce_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
132806,NSE,D,49492,OPTIDX,0,NIFTY-Jul2025-25100-CE,75.0,NIFTY 24 JUL 25100 CALL,2025-07-24 14:30:00,25100.0,CE,5.0,W,OP,NaN,NaN


In [6]:
filtered_pe_masterlist = masterlist[
    (masterlist.SEM_INSTRUMENT_NAME == 'OPTIDX') &
    (masterlist.SEM_TRADING_SYMBOL.str.startswith('NIFTY')) &
    (masterlist.SEM_EXPIRY_DATE == '2025-07-24 14:30:00') &
    (masterlist.SEM_STRIKE_PRICE == NIFTY_PREMIMUM) &
    (masterlist.SEM_OPTION_TYPE == 'PE')
]

filtered_pe_masterlist

,SEM_EXM_EXCH_ID,SEM_SEGMENT,SEM_SMST_SECURITY_ID,SEM_INSTRUMENT_NAME,SEM_EXPIRY_CODE,SEM_TRADING_SYMBOL,SEM_LOT_UNITS,SEM_CUSTOM_SYMBOL,SEM_EXPIRY_DATE,SEM_STRIKE_PRICE,SEM_OPTION_TYPE,SEM_TICK_SIZE,SEM_EXPIRY_FLAG,SEM_EXCH_INSTRUMENT_TYPE,SEM_SERIES,SM_SYMBOL_NAME
132807,NSE,D,49495,OPTIDX,0,NIFTY-Jul2025-25100-PE,75.0,NIFTY 24 JUL 25100 PUT,2025-07-24 14:30:00,25100.0,PE,5.0,W,OP,NaN,NaN


In [7]:

SECURITY_ID = 49492

IS_ACTIVE = False
IS_OPEN_POSITION = False
quantity = 0
buy_price = 0     
stop_loss = 0

while IS_ACTIVE is False:
        current_second = datetime.datetime.now().second
        time.sleep(1)
        print("waiting to activate", current_second)
        if current_second == 1:
            IS_ACTIVE = True

while IS_ACTIVE is True and IS_OPEN_POSITION is False:
      current_second = datetime.datetime.now().second
      if current_second == 0:
            break
      print('Active',current_second)
      time.sleep(0.15)
      intraday = dhan.intraday_daily_minute_charts(SECURITY_ID, dhan.NSE_FNO, "OPTIDX")
      print("intraday", intraday)
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

      for i in intraday_df['start_Time']:
          temp = dhan.convert_to_date_time(i)
          temp_list.append(temp)

      intraday_df['Date'] = temp_list

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]
      
      
     
      # if current_running_candle['close'] > previous_closed_candle['high']:
      #       print('Buy')
      #       length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      #       risk                      = 1000
      #       quantity                  = round(round(risk / length_of_previous_candle) / 75) * 75
      #       print('quantity',quantity)
            
      #       if quantity > 200:
      #             quantity = 200

      #       buy_price = int(current_running_candle['close'])
      #       stop_loss = int(previous_closed_candle['low'])

      #       dhan.place_order(
      #       security_id=SECURITY_ID,
      #       exchange_segment=dhan.NSE_FNO,
      #       transaction_type=dhan.BUY,
      #       quantity=quantity,
      #       order_type=dhan.MARKET,
      #       product_type=dhan.INTRA,
      #       price=0)  

      #       logging.info(f"BUY at {buy_price}")

      #       IS_OPEN_POSITION = True
      #       break;
      print('Buy')
      length_of_previous_candle = round(previous_closed_candle['high'] - previous_closed_candle['low'])
      risk                      = 1000
      quantity                  = 75 #round(round(risk / length_of_previous_candle) / 75) * 75
      print('quantity',quantity)
      
      if quantity > 200:
            quantity = 200

      buy_price = int(current_running_candle['close'])
      stop_loss = int(previous_closed_candle['low'])

      dhan.place_order(
      security_id=SECURITY_ID,
      exchange_segment=dhan.NSE_FNO,
      transaction_type=dhan.BUY,
      quantity=quantity,
      order_type=dhan.MARKET,
      product_type=dhan.INTRA,
      price=0)  

      logging.info(f"BUY at {buy_price}")

      IS_OPEN_POSITION = True
      break




waiting to activate 55
waiting to activate 56
waiting to activate 57
waiting to activate 58
waiting to activate 59
waiting to activate 0
waiting to activate 1
Active 2
intraday {'status': 'success', 'remarks': '', 'data': {'open': [74.85, 80.9, 79.1, 76.9, 79.6, 82.95, 87.05, 82.1, 81.9], 'high': [86.0, 85.45, 81.0, 81.25, 84.9, 92.6, 90.35, 83.95, 83.1], 'low': [72.95, 77.7, 75.3, 75.8, 79.5, 82.95, 82.15, 75.2, 81.9], 'close': [80.4, 77.95, 75.9, 78.7, 83.5, 86.85, 82.5, 82.05, 83.1], 'volume': [5450025.0, 3750000.0, 2159325.0, 2401275.0, 2621850.0, 5073825.0, 4445250.0, 3995775.0, 286800.0], 'start_Time': [1437709500.0, 1437709560.0, 1437709620.0, 1437709680.0, 1437709740.0, 1437709800.0, 1437709860.0, 1437709920.0, 1437709980.0]}}
Buy
quantity 75


In [ ]:

COST_TO_COST_SL = False
TRAILING_SL = False

while IS_OPEN_POSITION is True and quantity > 0:
      time.sleep(0.10)
      intraday = dhan.intraday_daily_minute_charts(SECURITY_ID, dhan.NSE_FNO, "OPTIDX")
      print("waiting for sell")
      intraday_df = pd.DataFrame(intraday['data']).tail(2)
      temp_list = []

      for i in intraday_df['start_Time']:
          temp = dhan.convert_to_date_time(i)
          temp_list.append(temp)

      intraday_df['Date'] = temp_list

      previous_closed_candle = intraday_df.iloc[-2]
      current_running_candle = intraday_df.iloc[-1]

      # Stop loss
      if current_running_candle['close'] < stop_loss and COST_TO_COST_SL is False and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  
            logging.info(f"SL hit {current_running_candle['close']}")

      # Trailing SL
      if current_running_candle['close'] < previous_closed_candle['low'] and COST_TO_COST_SL is False and TRAILING_SL is True:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  
            logging.info(f"Trailing SL hit {current_running_candle['close']}")

      # Shift sl to Cost to Cost
      if current_running_candle['close'] < buy_price +1 and COST_TO_COST_SL is True and TRAILING_SL is False:
            print("SELL")
            IS_OPEN_POSITION = False
          
           # Place an order for NSE Futures & Options
            dhan.place_order(security_id=SECURITY_ID,
            exchange_segment=dhan.NSE_FNO,
            transaction_type=dhan.SELL,
            quantity=quantity,
            order_type=dhan.MARKET,
            product_type=dhan.INTRA,
            price=0)  
            logging.info(f"CTC SL hit {current_running_candle['close']}")

            

waiting for sell
waiting for sell
waiting for sell
waiting for sell
waiting for sell
waiting for sell
waiting for sell


ValueError: DataFrame constructor not properly called!